## PCA: Principal Component Analysis

This notebook covers some of the details behind PCA. In particular this notebook covers the underlying math involved, and provides an implementation. If the linear algebra is a little confusing, feel free to contact me on Slack. Keep in mind, the linear algebra required for PCA will be covered in my notes. 

### PCA Formulation:





PCA is like any other machine learning algorithm, there is an associated cost/error function. Suppose that we have some data with $n$ features, and we have number of realisations of data given by $m$. This data would be stored as a ($m$ x $n$) matrix. The idea is to write each $n$ length piece of data as a linear combination of $k$ vectors. That way we only need to store the k vectors and the k coefficients for each data piece (Can be stored in a ($m$ x $k$) matrix.) We do this in a way such that the k vectors are chosen to minimise this cost function. Minimising this cost function, ensures that this smaller ($m$ x $k$) matrix, represents our data as closely as possible.

Formally, If $x_1, x_2, ...., x_m$ is our data vectors, we want to find $k$ orthonormal vectors: $ \{ v_1, v_2, ...., v_k \} $ such that if $P$ denotes the orthogonal projection of vectors onto the space $V := Span \{ v_1, v_2, ...., v_k \}$, then the following cost function is minimised:

$$ \sum^{m}_{i=1} ||x_i - P(x_i)||^2 $$

Assume we want to utilise $k$ components for our PCA algorithm. Recall that PCA is an approximation of our data. If we try to retrieve our data from the PCA matrix, this can be quite different to the original data matrix. Utilising more components in our algorithm will improve the accuracy of our approximation but consequently requires more work. 

Notation: 
- $X$ = Matrix of Data
- $k$ = Number of Components used for PCA
- $m$ = Number of data, ie, number of rows of $X$. 



There are 6 main steps in PCA(k - Components):

1. Normalise your data, this isn't always necessary but provides an improvement in terms of optimisation. 

2. Compute the covariance matrix of the data.

 $ \hspace{5cm} \Sigma := \frac{1}{m} X^{T}X $ = Covariance Matrix 

3. Compute eigenvalues of the covariance matrix, we need to sort the eigenvalues and take the $k$ largest eigenvalues and their corresponding eigenvectors. 
4. Form a new matrix (denote by $U_{reduced}$), with $k$-columns, each column given by an eigenvector, the eigenvectors should be arranged such that the eigenvectors with larger absolute values are positioned to the left of the matrix. 

Formally, let : $ e_1, e_2, e_3, .... e_k $ be the $k$ eigenvectors for the $k$ largest eigenvalues, then:

$$ U_{reduced} = \begin{pmatrix} e_1, e_2, e_3, ..... e_k \end{pmatrix} \in \mathbb{R}^{m\text{x}k} $$

5. The columns of this new matrix is what forms our orthonomal basis of the subspace that we are projecting onto. The projection theorem states that, if P is the orthongal projection onto a $k$-dimensional subspace, where the projection distance is minimised, then we can write the projection of x as equation (1.1). 

$$ P(x) = \sum^{k}_{i=1} \langle x, e_i \rangle e_i \hspace{5cm} (1.1) $$

6. Thus from equation (1.1) it follows that the $m$ x $k$ matrix or what I call the PCA matrix is given by: $U_{reduced}X  \in \mathbb{R}^{m\text{x}k}$


## Code:

In [25]:
import numpy as np 
from sympy.interactive import printing 
printing.init_printing(use_latex = True)
import os 


In [26]:
def normalise(matrix):
    """
    A function to normalise the data we have.

    Parameters:
        matrix (numpy array): Matrix to be normalised.

    Returns:
        normalised_matrix (numpy array): normalised matrix
        mean_vec (numpy array) : a numpy list of column means of the matrix provided.
        std (numpy array) : a numpy list of column standard deviations of the matrix provided. 
    
    
    """
    number_of_data = matrix.shape[1]
    mean_vec = matrix.mean(axis=0)
    std = matrix.std(axis=0)
    mean_matrix = np.einsum('ij,j-> ij', np.ones((number_of_data,number_of_data)), mean_vec)
    normalised_matrix = matrix - mean_matrix 
    try:
        normalised_matrix = np.einsum('ij, j-> ij', normalised_matrix, np.reciprocal(std))
        return normalised_matrix, mean_vec, std  
    except:
        raise ValueError('Atleast one standard deviation is 0')
    


def pca(design_matrix, k):
    """
    A function to apply the PCA algorithm.

    Parameters:
        design_matrix (numpy array): A matrix of our data, should be normalised using the "normalise" function. 
        k (int): Integer representing the number of components we want to use. 

    Returns:
        pca_matrix (numpy array): A matrix of coefficients representing our data.
        k_components (numpy array): A matrix whose columns are the eigenvectors of the covariance matrix. 
        percent_retained (float): A float from 0-100, representing the variance of the data captured. 
    
    
    """

    if k > design_matrix.shape[1]:
        raise ValueError('The space you want to project to must have a lower dimension than the data')

    else:
        # Compute Covariance Matrix
        m = design_matrix.shape[0]
        covariance_matrix = 1/m * np.matmul(design_matrix.transpose(), design_matrix)


        # Compute Eigenvalues, Eigenvectors
        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)


        idx_for_sort = eigenvalues.argsort()[::-1]
        eigenvalues_sorted = eigenvalues[idx_for_sort]
        eigenvectors_sorted = eigenvectors[:,idx_for_sort]

        # Take the first K-eigenvectors, these form an orthonormal basis of the Hyperplane we are projecting onto. 
        k_components = eigenvectors_sorted[:,:k]

        pca_matrix = np.matmul(design_matrix, k_components)

        percent_retained = eigenvalues_sorted[:k].sum()/eigenvalues_sorted.sum() * 100

        return pca_matrix, k_components, percent_retained


def retrieve_data(pca, components):
    """
    Given our PCA matrix and the components, this function will retrieve an approximation of our original data. 

    Parameters:
        pca (numpy array): Matrix of coefficients that represent our data.
        components(numpy array): A matrix whose columns are the basis of the space we projected our data into. 

    Returns: 
        data (numpy array): An approximation of the original data. 
    
    
    """
    data = np.matmul(pca, components.transpose())
    return data 
    

def un_normalise(normalised_matrix, mean, sd):
    """
    A function to unormalise our data. 

    Parameters:
        normalised_matrix (numpy array): Matrix to be un-normalised. 
        mean (numpy array): List of means we want un-normalise with.
        sd (numpy array): list of standard deviations we want to un-normalise with. 
    
    Returns:
        un_normalised_data (numpy array): An unormalised matrix of data. 
    
    
    """
    number_of_data = normalised_matrix.shape[1]
    mean_matrix = np.einsum('ij,j-> ij', np.ones((number_of_data, number_of_data)), mean)
    un_normalised_data = np.einsum('ij, j->ij', normalised_matrix, sd) + mean_matrix 
    return approximate_data 

### Example: Data Compression:

A practical example of PCA is data compression. PCA involves reducing the dimension of our $(m\text{x}n)$ data matrix into a $(m\text{x}k)$ matrix. We can store the data entirely as a $(m\text{x}n)$ matrix, or we can store the $(m\text{x}k)$ along with the $k$ eigenvectors, and when we need to use our data, we can retrive an approximation of our original data. 

### Retrieving Data:

Notation: 
- $X_{approx}$ = Approximation of Data
- $U_{reduced}$ = Same as above.
- $Y$ = Output of PCA algorithm, ie, the $(m\text{x}k)$ matrix. 


$$ X_{approx} = Y U_{reduced}^T $$


Now that we know how to retrieve an approximation of our data. An application is shown below. We generate a random matrix with 5000 rows and 2000 columns and store as a CSV. Then we apply the PCA algorithm, store the PCA matrix and it's component matrix into a CSV file and compare the file sizes:

In [27]:
random_mat = np.random.random((5000,2000))
np.savetxt('random_mat.csv', random_mat,delimiter=',')

pca_data = pca(random_mat, 600)
np.savetxt('PCA.csv', pca_data[0], delimiter=',')
np.savetxt('K_Components.csv', pca_data[1], delimiter=',')

retrieved_data = retrieve_data(pca_data[0], pca_data[1])

print(f'Storage used for entire matrix: {round(os.path.getsize("./random_mat.csv")/1000000,ndigits=2)} Bytes')
print(f'Storage used for reduced matrix and matrix of eigenvectors: {round(os.path.getsize("./K_Components.csv")/1000000 + os.path.getsize("./PCA.csv")/1000000, ndigits=2)} Bytes')


Storage used for entire matrix: 250.0 Bytes
Storage used for reduced matrix and matrix of eigenvectors: 107.1 Bytes


We see that storing the PCA matrix and the components matrix uses less than half the storage. 